In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [3]:
class InputEmbedding(nn.Module):
    def __init__(self, d_model: int , vocab_size: int ):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)
        
    def forward(self , x):
        return self.embedding(x)*math.sqrt(self.d_model)
        

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self , d_model:int, seq_len: int , dropout: float) -> None:
        
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        #creation of matrix shape (seq_len(row) , d_model(column))
        
        pe = torch.zeros(seq_len, d_model)
        
        #creation of position vector of shape(seq_len,1)
        
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) #pos
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # 2i/10000^d_model
        
        #applying sin to even indices and cos to odd indicies
        
        pe[: , 0::2] =  torch.sin(position * div_term)
        pe[: , 1::2] =  torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0) # (1,seq_len,d_model)
        
        self.reister_buffer('pe' , pe)
        
    def forward(self, x):
        x = x + (self.pe[ : , : x.shape[1], : ]).requires_grad_(False)
        return self.dropout(x)
        

In [4]:
class selfattention(nn.Module):
    
    def __init__(self, d_model =2 , row_dim = 0, col_dim = 1):
        super().__init__()
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias = False)
        self.W_k=  nn.Linear(in_features=d_model, out_features=d_model, bias = False)
        self.W_v=  nn.Linear(in_features=d_model, out_features=d_model, bias = False)
        
        self.row_dim = row_dim
        self.col_dim = col_dim
        
    def forward(self, token_embedding):
        x = token_embedding
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)
        
        #(q*k^T)
        
        sims = torch.matmul(q,k.transpose(dim0 = self.row_dim,dim1 =  self.col_dim))
        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)
        
        attention_percents = F.softmax(scaled_sims, dim = self.col_dim)
        
        attention_score = torch.matmul(attention_percents, V)
        
        return attention_score
        
        
        